In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from datetime import datetime
from tools import *
import numpy as np
import pyvisa

In [ ]:
rm = pyvisa.ResourceManager()            # Resource Manager
resource = rm.list_resources()[0]        # List Resources
sourcemeter = rm.open_resource(resource) # Open Conection with Device
device = sourcemeter.query('*IDN?')      # Device

print(f'GPIB-USB-HS: {resource}')
print(f'     Device: {device}')

In [ ]:
# Set Current
I = current_(1.0e-7, 5.0e-6, 100)

In [ ]:
# Mesurement
sourcemeter.write('*RST')               # Reset Device
sourcemeter.write('SENS:FUNC "VOLT"')   # Mesure --> Voltage
sourcemeter.write('SENS:VOLT:RSEN ON')  # Set 4-Wire Mesure
sourcemeter.write('SOUR:FUNC CURR')     # Source --> Current
sourcemeter.write('OUTP ON')            # Turn-on Output
V = voltage_(sourcemeter, I, delay=0.1) # Voltage
sourcemeter.write('OUTP OFF')           # Turn-off Output
sourcemeter.close()                     # Close Conection with Device

In [ ]:
# Resistence and Resual Voltage
model = LinearRegression()
model.fit(I.reshape(-1,1), V)

R = model.coef_[0]    # Resistence
V0 = model.intercept_ # Resual Voltage
print(f' R: {R:10.4e} Ω')
print(f'V0: {V0:10.4e} V')

In [ ]:
# Plot IvsV
fig, ax = plt.subplots(1,1, figsize=(2.5,2.5))

ax.scatter(1.0e6*I, 1.0e3*V, color='brown', s=1)
ax.set_xlabel('$Current$ (μA)', fontsize=9)
ax.set_ylabel('$Voltage$ (mV)', fontsize=9)
ax.set_title(f'$R={R:.2f}$ Ω', fontsize=8)

plt.show()

In [ ]:
# Save Data and Figure
date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
np.savetxt(f'./Data/IvsV_{date}.txt', np.c_[I, V], header='        I (A)           V (V)', fmt=['%15.8e', '%15.8e'])
fig.savefig(f'./Data/IvsV_{date}.pdf', dpi=300, pad_inches=0.1, bbox_inches='tight')